In [1]:
import torch
import numpy as np
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import math

In [2]:
class SubstringDataset(Dataset):
    LETTERS = list('cpen')

    def __init__(self, seed, dataset_size, str_len=20):
        super().__init__()
        self.str_len = str_len
        self.dataset_size = dataset_size
        self.rng = np.random.default_rng(seed)
        self.strings, self.labels = self._create_dataset()

    def __getitem__(self, index):
        return self.strings[index], self.labels[index]

    def __len__(self):
        return self.dataset_size

    def _create_dataset(self):
        strings, labels = [], []
        for i in range(self.dataset_size):
            label = i%2
            string = self._generate_random_string(bool(label))
            strings.append(string)
            labels.append(label)
        return strings, labels
    
    def _generate_random_string(self, has_cpen):
        while True:
            st = ''.join(self.rng.choice(SubstringDataset.LETTERS, size=self.str_len))
            if ('cpen' in st) == has_cpen:
                return st

In [ ]:
# vocab = {
#             '[CLS]': 0,
#             'c': 1,
#             'p': 2,
#             'e': 3,
#             'n': 4,
#         }
# char_list = ['c', 'c', 'p']
# for i in range(len(char_list)):
#   char_list[i] = vocab[char_list[i]]
# char_list.insert(0, 0)
# print(char_list)


In [ ]:
# string = 'cccp'
# x = list(string)
# print(x)

In [ ]:
# tokenized_string = []
# for c in char_list:
#   arr = list(np.zeros(len(vocab), dtype = int))
#   arr[c] = 1
#   tokenized_string.append(arr)

# print(tokenized_string)

In [3]:
class Tokenizer():
    def __init__(self) -> None:
        self.vocab = {
            '[CLS]': 0,
            'c': 1,
            'p': 2,
            'e': 3,
            'n': 4,
        }


    def tokenize_string(self, string, add_cls_token=True) -> torch.Tensor:
        """
        Tokenize the input string according to the above vocab

        START BLOCK
        """
        #1.splits the string and returns a list of characters(tokens)
        char_list = list(string)
        #print(char_list) ## remove before submit
        #1-1. change char into num in vocab
        for i in range(len(char_list)):
          char_list[i] = self.vocab[char_list[i]]
        #2. add a [CLS] token to the beginning of the list
        if (add_cls_token == True): #add [CLS]
          char_list.insert(0, 0)
        
        #3. convert each token into a one-hot vector and return resulting matrix
        tokenized_string = torch.zeros((len(char_list),len(self.vocab)))
        for i in range(len(char_list)):
          arr = list(torch.zeros(len(self.vocab)))
          arr[char_list[i]] = 1
          tokenized_string[i,:] = torch.tensor(arr)
        # print(tokenized_string.shape)
        # print(tokenized_string)
        #tokenized_string = None #
        """
        END BLOCK
        """
        return tokenized_string

    def tokenize_string_batch(self, strings, add_cls_token=True):
        X = []
        for s in strings:
            X.append(self.tokenize_string(s, add_cls_token=add_cls_token))
        return torch.stack(X, dim=0)



In [ ]:
# W = torch.empty(3,4)
# print(W.shape)

# p_W = W[0:2, :]
# print(p_W.shape)

In [ ]:
# H = [torch.empty((3, 4)) for _ in range(2)]
# print(H)
# print(H[0].shape)
# print(H[1].shape)

In [ ]:
# a = torch.ones((2,3))
# b = torch.ones((2,3))
# c = torch.concat((a,b),0)
# print(c.shape)
# c = torch.concat((a,b),1)
# print(c.shape)

In [ ]:
# a = torch.randn(2, 3)
# print(a)
# soft = F.softmax(a, dim = 1)
# print(soft)

# soft = F.softmax(a, dim = 0)
# print(soft)

In [ ]:
# mAX_LEN = 2
# a = torch.empty((2*mAX_LEN+1, ))
# print(a)
# print(a.shape)

In [ ]:
# for i in range(2):
#   print(i)

# t = [1,2,3,4,5]
# print(t[0:5])

In [4]:
class AbsolutePositionalEncoding(nn.Module):
    MAX_LEN = 256 # N(length of Sequence) <= MAX_LEN
    def __init__(self, d_model): #d_model: the number of expected features in the encoder/decoder inputs (default=512)
        super().__init__()
        self.W = nn.Parameter(torch.empty((self.MAX_LEN, d_model))) 
        #i-th row: learnable vector corresponding to the i-th position in a sequence
        nn.init.normal_(self.W)
    """This module applies positional encoding to a sequence 
    by element-wise adding rows of this matrix to their corresponding position in the input."""

    def forward(self, x):
        """
        args:
            x: shape B x N x D # N: length of Sequence. B: Batch size. D: After tokenizer, each char becomes D-dimensional one-hot vector
        returns:
            out: shape B x N x D # D = d_model...?

        START BLOCK
        """
        # print("AbsolutePositional Encoding")
        # print("W shape is ", self.W.shape)
        # print(x.shape)
        B,N,D = x.shape
        out = torch.zeros_like(x)

        part_W = self.W[0:N, :]
        for i in range(B):
          out[i, :, :] = torch.add(part_W,x[i, :, :])

        # out = None
        """
        END BLOCK
        """
        return out

class MultiHeadAttention(nn.Module):
    MAX_LEN = 256

    def __init__(self, d_model, n_heads, rpe):
        super().__init__()
        assert d_model % n_heads == 0, "Number of heads must divide number of dimensions"
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_h = d_model // n_heads
        self.rpe = rpe #boolean 
        self.Wq = nn.ParameterList([nn.Parameter(torch.empty((d_model, self.d_h))) for _ in range(n_heads)])
        self.Wk = nn.ParameterList([nn.Parameter(torch.empty((d_model, self.d_h))) for _ in range(n_heads)])
        self.Wv = nn.ParameterList([nn.Parameter(torch.empty((d_model, self.d_h))) for _ in range(n_heads)])
        self.Wo = nn.Parameter(torch.empty((d_model, d_model)))
       
        if rpe:
            # -MAX_LEN, -MAX_LEN+1, ..., -1, 0, 1, ..., MAX_LEN-1, MAXLEN
            self.rpe_w = nn.ParameterList([nn.Parameter(torch.empty((2*self.MAX_LEN+1, ))) for _ in range(n_heads)])

        for h in range(self.n_heads):
            nn.init.xavier_normal_(self.Wk[h])
            nn.init.xavier_normal_(self.Wq[h])
            nn.init.xavier_normal_(self.Wv[h])
            if rpe:
                nn.init.normal_(self.rpe_w[h])
        nn.init.xavier_normal_(self.Wo)

    def forward(self, key, query, value):
        """
        args:
            key: shape B x N x D
            query: shape B x N x D
            value: shape B x N x D
        return:
            out: shape B x N x D

        START BLOCK
        """
        B,N,D = key.shape
        out = torch.zeros_like(key)
        tmp = torch.zeros_like(key)
        m = self.MAX_LEN

        # print(self.d_h) #32 (w/o RPE)
        # print(N) #12 (w/o RPE)
        # nn.ParameterList([nn.Parameter(torch.empty((d_model, self.d_h))) for _ in range(n_heads)])
        #H = nn.ParameterList([nn.Parameter(torch.empty((D, self.d_h))) for _ in range(self.n_heads)])
        H = [torch.empty((D,self.d_h)).to(key.device) for _ in range(self.n_heads)]
        #print(H[0].shape)
        #H = H.to(key.device)
        #print("H[0] ", H[0].is_cuda)

        # TO DO for RPE
        #print("Wq[0]", self.Wq[0].is_cuda)
        #self.Wq = nn.ParameterList([nn.Parameter(torch.empty((d_model, self.d_h))) for _ in range(n_heads)])
        #M = nn.ParameterList([nn.Parameter(torch.empty((N,N))) for _ in range(self.n_heads)]) #list of matrix
        M = [torch.zeros((N,N)).to(key.device) for _ in range(self.n_heads)]
        #M = M.to(key.device)
        # for h in range(self.n_heads):
        #     nn.init.xavier_normal_(M[h])
        #print("M[0] ", M[0].is_cuda)
        if self.rpe:
          for h in range(self.n_heads): #h: head index number
            degree_freedom = self.rpe_w[h][m-(N-1):m+(N-1)+1] #(indexing. 2N-1개)
            #Use degree_freedom list to make Toeplitz matrix
            for i in range(N):
              temp_vec = degree_freedom[(N-1-i):(N-1-i+N)]
              M[h][i,:] = temp_vec
       
        # print(self.rpe)

        for b in range(B):
          for h in range(self.n_heads):
            XWq = torch.matmul(query[b,:,:],  self.Wq[h])
            #print("XWq shape is ", XWq.shape) #(n, self.d_h) = (12, 32)
            XWk_t = torch.transpose(torch.matmul(key[b,:,:], self.Wk[h]),0,1)
            #print("XWk_t shape is ", XWk_t.shape) #(self.d_h, n) = (32, 12)
            matmul_XWq_XWk_t = torch.matmul(XWq, XWk_t)
            #before softmax, M[h] element-wise addition 
            # print("matmul_XWq_XWk_t ", matmul_XWq_XWk_t.is_cuda)
            # print("M[h] ", M[h].is_cuda)
            matmul_XWq_XWk_t = matmul_XWq_XWk_t + M[h]
            #print("matmul_XWq_XWk_t shape is ", matmul_XWq_XWk_t.shape) #(n, n) = (12,12)
            div_root_dh = matmul_XWq_XWk_t / math.sqrt(self.d_h)
            #print("div_root_dh shape is ", div_root_dh.shape) #(n, n) = (12,12)
            soft = F.softmax(div_root_dh, dim = 1)
            #print("soft shape is ", soft.shape) # (n, n) = (12,12)
            XWv = torch.matmul(value[b,:,:], self.Wv[h])
            #print("XWv shape is ", XWv.shape)  #(n, self.d_h)  = (12, 32)
            H[h] = torch.matmul(soft, XWv)
            #print(" H[h] shape is ",  H[h].shape) #(n, self.d_h)  = (12, 32)
          #print(H[0].shape) # n, d_h = 12, 32
          out[b,:,:] = torch.concat([H[h]for h in range(self.n_heads)],1)
          out[b,:,:] = torch.matmul(out[b,:,:].clone(),self.Wo)
        # out = None
        """
        END BLOCK
        """
        return out


In [6]:
class TransformerLayer(nn.Module): #transformer encoder layer
    def __init__(self, d_model: int, n_heads: int, prenorm: bool, rpe: bool):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.prenorm = prenorm #boolean
        self.attention = MultiHeadAttention(d_model, n_heads, rpe=rpe)
        self.fc_W1 = nn.Parameter(torch.empty((d_model, 4*d_model)))
        self.fc_W2 = nn.Parameter(torch.empty((4*d_model, d_model)))
        self.relu = nn.ReLU()
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)

        nn.init.xavier_normal_(self.fc_W1)
        nn.init.xavier_normal_(self.fc_W2)
    
    def forward(self, x):
        """
        args:
            x: shape B x N x D (B: batch_size, N: length of sequence, D = d_model )
        returns:
            out: shape B x N x D

        START BLOCK
        """
        B,N,D = x.shape
        # print("transformer layer")
        # print(x.is_cuda)

        if self.prenorm: #prenorm
          after_ln1 = self.ln1(x) #(B,N,D)
          mha_result = self.attention(after_ln1, after_ln1, after_ln1) #Xk=Xq=Xv=X #(B,N,D)
          add_1 = x + mha_result #(B,N,D)

          after_ln2 = self.ln2(add_1) #(B,N,D)

          # after_fc1 = torch.ones(B,N,4*D)
          # after_fc2 = torch.ones(B, N, D)

          # for b in range(B):
          #   after_fc1[b,:,:] = torch.matmul(after_ln2[b,:,:],  self.fc_W1)
          #   after_relu = self.relu(after_fc1[b,:,:])
          #   after_fc2[b,:,:] = torch.matmul(after_relu, self.fc_W2)
          
          ###
          after_fc1 = torch.matmul(after_ln2,  self.fc_W1)
          after_relu = self.relu(after_fc1)
          after_fc2 = torch.matmul(after_relu, self.fc_W2)
          ###

          add_2 = add_1 + after_fc2
          out = add_2


        else: #postnorm
          mha_result = self.attention(x, x, x) #Xk=Xq=Xv=X #(B,N,D)
          add_1 = x + mha_result
          after_ln1 = self.ln1(add_1)

          # after_fc1 = torch.ones(B,N,4*D)
          # after_fc2 = torch.ones(B, N, D)

          # for b in range(B):
          #   after_fc1[b,:,:] = torch.matmul(after_ln1[b,:,:],  self.fc_W1)
          #   after_relu = self.relu(after_fc1[b,:,:])
          #   after_fc2[b,:,:] = torch.matmul(after_relu, self.fc_W2)

          ####
          after_fc1= torch.matmul(after_ln1,  self.fc_W1)
          after_relu = self.relu(after_fc1)
          after_fc2 = torch.matmul(after_relu, self.fc_W2)


          ###
          
          add_2 = after_ln1 + after_fc2
          after_ln2 = self.ln2(add_2)
          out = after_ln2
      
        """
        END BLOCK
        """
        return out

In [ ]:
# a = torch.zeros((1,2,3))
# b = torch.zeros((2,2,3))

# print(a[0, :, :].shape)

In [7]:
class ModelConfig:
    n_layers = 4
    input_dim = 5
    d_model = 256
    n_heads = 4
    prenorm = True
    pos_enc_type = 'ape' # 'ape': Abosolute Pos. Enc., 'rpe': Relative Pos. Enc.
    output_dim = 1 # Binary output: 0: invalid, 1: valid

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            assert hasattr(self, k)
            self.__setattr__(k, v)

class TransformerModel(nn.Module): #In this assignment, you only have TransformerEncoder (w/ just encoder, decoder) So, technically, you are still implementing an actual transformer, but a special variant..
    def __init__(self, cfg: ModelConfig):
        super().__init__()
        self.cfg = cfg
        self.enc_W = nn.Parameter(torch.empty((cfg.input_dim, cfg.d_model)))
        if cfg.pos_enc_type == 'ape':
            self.ape = AbsolutePositionalEncoding(d_model=cfg.d_model)
        self.transformer_layers = nn.ModuleList([
            TransformerLayer(d_model=cfg.d_model, n_heads=cfg.n_heads, prenorm=cfg.prenorm, rpe=cfg.pos_enc_type == 'rpe') for _ in range(cfg.n_layers)
        ])
        self.dec_W = nn.Parameter(torch.empty((cfg.d_model, cfg.output_dim)))

        nn.init.xavier_normal_(self.enc_W)
        nn.init.xavier_normal_(self.dec_W)
    
    def forward(self, x):
        """
        args:
            x: shape B x N x D_in
        returns:
            out: shape B x N x D_out

        START BLOCK
        """
        # print("transformer model")
        # print(x.is_cuda)
        #print("x is", x)
        B,N,D_in = x.shape
        D_out = self.cfg.output_dim
        D = self.cfg.d_model
        
        after_encoder = torch.matmul(x, self.enc_W)
        # print("after_encoder")
        # print(after_encoder.is_cuda)
        if self.cfg.pos_enc_type == 'ape':
          after_encoder = self.ape(after_encoder)
          # print("ape")
          # print(after_encoder.is_cuda)

        ###
        # after_1 = self.transformer_layers[0](after_encoder)
        # after_2 = self.transformer_layers[1](after_1)
        # after_3 = self.transformer_layers[2](after_2)
        # after_4 = self.transformer_layers[3](after_3)

        # after_decoder = torch.zeros(B,N,D_out)
        # for b in range(B):
        #   after_decoder[b,:,:] = torch.matmul(after_4[b,:,:], self.dec_W)
        ###
        
        for i in range(self.cfg.n_layers):
          after_encoder = self.transformer_layers[i](after_encoder)
        
        after_decoder = torch.matmul(after_encoder, self.dec_W)

        out = after_decoder

        # print("afterdecoder cuda")
        # print(after_decoder.is_cuda)

        """
        END BLOCK
        """
        return out

In [8]:
from torch.optim import lr_scheduler

class CustomScheduler(lr_scheduler._LRScheduler):
    def __init__(self, optimizer, total_steps, warmup_steps=1000):
        self.total_steps = total_steps #maxSTEP
        self.warmup_steps = warmup_steps #warmupSTEP
        super().__init__(optimizer)

    def get_lr(self):
        """
        Compute the custom scheduler with warmup and cooldown
        Hint: self.last_epoch contains the current step number

        START BLOCK
        """
        # print(self.optimizer)
        # print(self.last_epoch)
        # base_lr = self.optimizer.param_groups[0]['initial_lr']
        #print("base lr is ", self.optimizer.param_groups[0]['initial_lr'])

        current_step = self.last_epoch
        if (current_step <= self.warmup_steps): #increase (warmup)
          mult_factor = current_step/self.warmup_steps
          
        else: #decrease(cooldown)
          mult_factor = ((-1)/(self.total_steps - self.warmup_steps)) * current_step + self.total_steps/(self.total_steps - self.warmup_steps)
          
        
        """
        END BLOCK
        """
        return [group['initial_lr'] * mult_factor for group in self.optimizer.param_groups]

In [9]:
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class TrainerConfig:
    lr = 0.003
    train_steps = 5000
    batch_size = 256
    evaluate_every = 100
    device = 'cpu'

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            assert hasattr(self, k)
            self.__setattr__(k, v)

class Trainer:
    def __init__(self, model, cfg: TrainerConfig):
        self.cfg = cfg
        self.device = cfg.device
        self.tokenizer = Tokenizer()
        self.model = model.to(self.device)

    def train(self, train_dataset, val_dataset):
        ##
        # for param in self.model.parameters():
        #   print(param, param.requires_grad)
        ##
        optimizer = optim.Adam(self.model.parameters(), lr=self.cfg.lr)
        scheduler = CustomScheduler(optimizer, self.cfg.train_steps)
        train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=self.cfg.batch_size)
        # with torch.autograd.detect_anomaly():
        for step in range(self.cfg.train_steps):
            self.model.train()
            batch = next(iter(train_dataloader))
            strings, y = batch
            x = self.tokenizer.tokenize_string_batch(strings)

            optimizer.zero_grad()
            loss, _ = self.compute_batch_loss_acc(x, y)
            loss.backward()
            optimizer.step()
            scheduler.step()
            if step % self.cfg.evaluate_every == 0:
                val_loss, val_acc = self.evaluate_dataset(val_dataset)
                print(f"Step {step}: Train Loss={loss.item()}, Val Loss: {val_loss}, Val Accuracy: {val_acc}")

    def compute_batch_loss_acc(self, x, y):
        """
        Compute the loss and accuracy of the model on batch (x, y)
        args:
            x: B x N x D_in
            y: B
        return:
            loss, accuracy
        START BLOCK
        """
        # print(type(x))
        x_cuda = x.to(self.device)
        y_cuda = y.to(self.device)
        criterion = nn.BCELoss()
        #criterion = nn.BCEWithLogitsLoss() #BCEWithLogitsLoss BCELoss
        outputs = self.model(x_cuda) #outputs: BxNxD_out
        # print("outputs")
        # print(outputs.requires_grad)
        cls = outputs[:,0,0]#BxD_out = B*1
        cls = torch.sigmoid(cls) #probability of class1(invalid) (Binary output: 0: invalid, 1: valid)
        # print(cls)
        # print("cls", cls)
        # print("y_cuda", y_cuda)
        #cls = cls.type(torch.float32)
        y_cuda = y_cuda.type(torch.float32)
        
        loss = criterion(cls, y_cuda)
        # loss.requires_grad = True
        #loss = loss.type(torch.long)
        # print("loss", loss)
        # print(loss.requires_grad)

        correct_pred = torch.tensor([0.0])
        #print("len(cls) is ", len(cls))
        for i in range(len(cls)):
          pred = 1 if cls[i] > 0.5 else 0 #threshold = 0.5
          if y[i] == pred:
            correct_pred = correct_pred + 1
          else:
            correct_pred = correct_pred +0
        acc = correct_pred / len(cls)
        # acc = acc.type(torch.long)
        # loss, acc = torch.tensor([1.0]), torch.tensor([0.0])
        """
        END BLOCK
        """
        return loss, acc
    
    @torch.no_grad()
    def evaluate_dataset(self, dataset):
        self.model.eval()
        dataloader = DataLoader(dataset, shuffle=False, batch_size=self.cfg.batch_size)
        final_loss, final_acc = 0.0, 0.0
        for batch in dataloader:
            strings, y = batch
            x = self.tokenizer.tokenize_string_batch(strings)
            loss, acc = self.compute_batch_loss_acc(x, y)
            final_loss += loss.item() * x.size(0)
            final_acc += acc.item() * x.size(0)
        return final_loss / len(dataset), final_acc / len(dataset)
    

In [ ]:
# #dataset
# train_dataset = SubstringDataset(seed=1, dataset_size=10_000, str_len=16)
# train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=256)
# batch = next(iter(train_dataloader))
# strings, y = batch
# print(y) #0 or 1

In [10]:
"""
In case you were not successful in implementing some of the above classes,
you may reimplement them using pytorch available nn Modules here to receive the marks for part 1.8
If your implementation of the previous parts is correct, leave this block empty.
START BLOCK
"""


"""
END BLOCK
"""
def run_transformer():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    #print("device is ", device) # remove b/f submit
    model = TransformerModel(ModelConfig())
    trainer = Trainer(model, TrainerConfig(device=device))
    parantheses_size=16
    print("Creating datasets.")
    train_dataset = SubstringDataset(seed=1, dataset_size=10_000, str_len=parantheses_size)
    val_dataset = SubstringDataset(seed=2, dataset_size=1_000, str_len=parantheses_size)
    test_dataset = SubstringDataset(seed=3, dataset_size=1_000, str_len=parantheses_size)

    print("Training the model.")
    trainer.train(train_dataset, val_dataset)
    test_loss, test_acc = trainer.evaluate_dataset(test_dataset)
    print(f"Final Test Accuracy={test_acc}, Test Loss={test_loss}")

In [ ]:
run_transformer()

Creating datasets.
Training the model.
Step 0: Train Loss=2.958362102508545, Val Loss: 2.709890213012695, Val Accuracy: 0.5
Step 100: Train Loss=0.6839382648468018, Val Loss: 0.7943038592338562, Val Accuracy: 0.5
Step 200: Train Loss=0.8427833318710327, Val Loss: 0.8729402265548706, Val Accuracy: 0.5
Step 300: Train Loss=0.6841714978218079, Val Loss: 0.693966637134552, Val Accuracy: 0.5420000052452087
Step 400: Train Loss=0.6919990181922913, Val Loss: 0.7365768885612488, Val Accuracy: 0.5
Step 500: Train Loss=0.7070266008377075, Val Loss: 0.6892928490638733, Val Accuracy: 0.5320000009536743
Step 600: Train Loss=0.6800819635391235, Val Loss: 0.6869859461784362, Val Accuracy: 0.5440000052452088
Step 700: Train Loss=0.7146021723747253, Val Loss: 0.7387604341506958, Val Accuracy: 0.5209999976158142
Step 800: Train Loss=0.8186905384063721, Val Loss: 0.8280617690086365, Val Accuracy: 0.5
Step 900: Train Loss=0.49571675062179565, Val Loss: 0.6330121359825134, Val Accuracy: 0.6780000052452088


# Unit Tests

In [11]:
import random
import numpy as np

def seed_all():
    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)

class TransformerUnitTest:
    def __init__(self, gt_vars: dict, verbose=False):
        self.gt_vars = gt_vars
        self.verbose = verbose

    def test_all(self):
        self.test_tokenizer()
        self.test_ape()
        self.test_mha()
        self.test_transformer_layer()
        self.test_transformer_model()
        self.test_scheduler()
        self.test_loss()

    def test_tokenizer(self):
        seed_all()
        self.check_correctness(
            Tokenizer().tokenize_string('ccpeen', add_cls_token=True),
            self.gt_vars['tokenizer_1'],
            "Tokenization with cls class"
        )
        self.check_correctness(
            Tokenizer().tokenize_string('cpppencpen', add_cls_token=False),
            self.gt_vars['tokenizer_2'],
            "Tokenization without cls class"
        )

    def test_ape(self):
        seed_all()
        ape_result = AbsolutePositionalEncoding(128)(torch.randn((8, 12, 128)))
        self.check_correctness(ape_result, self.gt_vars['ape'], "APE")

    def test_mha(self):
        seed_all()
        mha_result = MultiHeadAttention(d_model=128, n_heads=4, rpe=False)(
            torch.randn((8, 12, 128)), torch.randn((8, 12, 128)), torch.randn((8, 12, 128))
        )
        self.check_correctness(
            mha_result,
            self.gt_vars['mha_no_rpe'],
            "Multi-head Attention without RPE"
        )
        mha_result_rpe = MultiHeadAttention(d_model=128, n_heads=8, rpe=True)(
            torch.randn((8, 12, 128)), torch.randn((8, 12, 128)), torch.randn((8, 12, 128))
        )
        self.check_correctness(
            mha_result_rpe,
            self.gt_vars['mha_with_rpe'],
            "Multi-head Attention with RPE"
        )
    
    def test_transformer_layer(self):
        seed_all()
        for prenorm in [True, False]:
            transformer_layer_result = TransformerLayer(
                d_model=128, n_heads=4, prenorm=prenorm, rpe=False
            )(torch.randn((8, 12, 128)))
            self.check_correctness(
                transformer_layer_result,
                self.gt_vars[f'transformer_layer_prenorm_{prenorm}'],
                f"Transformer Layer Prenorm {prenorm}"
            )

    def test_transformer_model(self):
        seed_all()
        transformer_model_result = TransformerModel(
            ModelConfig(d_model=128, prenorm=True, pos_enc_type='ape') 
        )(torch.randn((8, 12, 5)))
        self.check_correctness(
            transformer_model_result,
            self.gt_vars['transformer_model_result'],
            f"Transformer Model"
        )

    def test_scheduler(self):
        model = TransformerModel(ModelConfig()) #original code
        #model = nn.Transformer(ModelConfig())
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        scheduler = CustomScheduler(optimizer, 10_000)
        optimizer.step()
        scheduler.step(521)
        self.check_correctness(
            torch.tensor([optimizer.param_groups[0]['lr']]),
            self.gt_vars['scheduler_1'],
            f"Scheduler Warmup"
        )
        scheduler.step(2503)
        self.check_correctness(
            torch.tensor([optimizer.param_groups[0]['lr']]),
            self.gt_vars['scheduler_2'],
            f"Scheduler Cooldown"
        )

    def test_loss(self):
        seed_all()
        model = TransformerModel(ModelConfig()) # original code
        # model = nn.Transformer(ModelConfig())
        trainer = Trainer(model, TrainerConfig(device='cpu'))
        loss_result, _ = trainer.compute_batch_loss_acc(
            torch.randn((8, 12, 5)),
            torch.ones(8).float(),
        )
        self.check_correctness(
            loss_result,
            self.gt_vars['loss'],
            f"Batch Loss"
        )

    def check_correctness(self, out, gt, title):
        try:
            diff = (out - gt).norm()
            # print("out", out)
            # print("gt",gt)
            #print(diff)
            # print(out.shape)
            # print(gt.shape)
            # print(out)
            # print(gt)
            # print(diff)
            #print(diff)
        except:
            diff = float('inf')
        if diff < 1e-4: # increase the epsilon from 1e-5 to 1e-4
            print(f"[Correct] {title}")
        else:
            print(f"[Wrong] {title}")
            if self.verbose:
                print("-----")
                print("Expected: ")
                print(gt)
                print("Received: ")
                print(out)
                print("-----")


In [12]:
!gdown 1-2-__6AALEfqhfew3sJ2QiCE1-rrFMnQ -q -O unit_tests.pkl
import pickle
with open('unit_tests.pkl', 'rb') as f:
    gt_vars = pickle.load(f)

In [13]:
TransformerUnitTest(gt_vars, verbose=False).test_all()

[Correct] Tokenization with cls class
[Correct] Tokenization without cls class
[Correct] APE
[Correct] Multi-head Attention without RPE
[Correct] Multi-head Attention with RPE
[Correct] Transformer Layer Prenorm True
[Correct] Transformer Layer Prenorm False
[Correct] Transformer Model
[Correct] Scheduler Warmup
[Correct] Scheduler Cooldown
[Correct] Batch Loss


/usr/local/lib/python3.9/dist-packages/torch/optim/lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
